In [116]:
#'random_500/random_result'
files = ['filtered_2_1173/fiiltered_2_result',
         'filtered_500/filtered_result',
         'recoms_filtered_2_tfidf_10000(0)_result',
         'recoms_filtered_2_tfidf_10000(1000)_result',
         'recoms_filtered_2_tfidf_10000(2000)_result']
tweet_texts = {}
news_texts = {}

def parse_tweet(rows, idx):
    if not rows[idx][0].isdigit(): raise(Exception('Incorrect first row of tweet: %s' % idx))
    text = rows[idx]
    
    idx += 1
    #print idx
    
    while not rows[idx][0].isdigit():
        text += ' ' + rows[idx]
        idx += 1
    
    if not rows[idx][0].isdigit(): raise(Exception('Incorrect second row of tweet: %s' % idx))    
    tweet_id = int(rows[idx])
    idx += 1
        
    while rows[idx][0] != '\t':
        if rows[idx][:4] != 'http': raise(Exception('Incorrect row with link of tweet: %s' % idx))
        idx += 1
        
    tweet_texts[tweet_id] = text
    return idx, tweet_id


def parse_news(rows, idx):
    marked = False
    
    if rows[idx][0] != '\t': raise(Exception('News row started without tabulation: %s' % idx))    
    if rows[idx][1] == "!": marked = True
    text = rows[idx]
    idx += 1    
    
    if rows[idx][0] != '\t': raise(Exception('News row started without tabulation: %s' % idx))    
    if rows[idx][1:5] != 'http': raise(Exception('Incorrect row with link of news: %s' % idx))    
    link = rows[idx][1:]
    idx += 1
    
    #print rows[idx]
    if rows[idx][1] != "-": raise(Exception('Incorrect row with news separator: %s' % idx))        
    idx += 1
    
    if marked: news_texts[link] = text
    return idx, link if marked else None


def parse_all_news(rows, idx):
    news_link = None
    
    #print rows[idx] == '\t'
    while idx < len(rows) and rows[idx] and rows[idx][0] == '\t':
        idx, link = parse_news(rows, idx)
        if link and not news_link:
            news_link = link        
        
    return idx, news_link

tweet_to_news = {}
for file_suffix in files:
    filepath = '/home/avybornov/twnews_data_april/dataset_markdown/' + file_suffix
    print filepath
    rows = open(filepath).read().splitlines()
    
    tweet_to_news[file_suffix] = {}
    idx = 0
    while idx < len(rows):
        idx, tweet_id = parse_tweet(rows, idx)
        idx, news_link = parse_all_news(rows, idx)
        tweet_to_news[file_suffix][tweet_id] = news_link
        
        #print idx
#         if idx > 40:
#             break

/home/avybornov/twnews_data_april/dataset_markdown/filtered_2_1173/fiiltered_2_result
/home/avybornov/twnews_data_april/dataset_markdown/filtered_500/filtered_result
/home/avybornov/twnews_data_april/dataset_markdown/recoms_filtered_2_tfidf_10000(0)_result
/home/avybornov/twnews_data_april/dataset_markdown/recoms_filtered_2_tfidf_10000(1000)_result
/home/avybornov/twnews_data_april/dataset_markdown/recoms_filtered_2_tfidf_10000(2000)_result


IndexError: string index out of range

In [117]:
already_marked = set([])

linked_for_files = {}

for file_suffix, data in tweet_to_news.iteritems():
    print file_suffix, len(data)
    linked = {}
    for tweet_id, link in data.items():
        already_marked.add(tweet_id)
        if link: linked[tweet_id] = link
    linked_for_files[file_suffix] = linked
    print 'linked: %s %.2f%%' % (len(linked), len(linked)*100.0/len(data))
        #print '\t', 'https://twitter.com/statuses/%s' % tweet_id, link
        
from twnews.utils.memoize import dump
dump(already_marked, 'already_marked')


filtered_2_1173/fiiltered_2_result 1173
linked: 254 21.65%
recoms_filtered_2_tfidf_10000(0)_result 1000
linked: 172 17.20%
recoms_filtered_2_tfidf_10000(1000)_result 1000
linked: 150 15.00%
recoms_filtered_2_tfidf_10000(2000)_result 1000
linked: 168 16.80%
filtered_500/filtered_result 500
linked: 261 52.20%


In [118]:
for file_suffix, data in linked_for_files.items()[2:3]:
    print file_suffix
    for tweet_id, link in data.items()[:20]:
        print '\t',tweet_texts[tweet_id]
        print '\t',news_texts[link]

#print len(already_marked)

recoms_filtered_2_tfidf_10000(1000)_result
	273) #fyseo_ru #rus #ruff Аргентина определила дату выпуска новых облигаций  #взаимный #Ячитаю
		!Аргентина определила дату выпуска новых облигаций и выплаты кредиторам
	788) "Пежо Ситроен Рус" отзывает 367 автомобилей из России  
		!"Пежо Ситроен Рус" отзывает 367 автомобилей из России
	216) В Москве начались показы фестиваля Perform
		!В Москве стартует международный кинофестиваль Perform
	388) Депутаты Госдумы РФ 12 апреля одобрили в первом чтении законопроект, регулирующий деятельность коллекторов и коллекторских агентств. Текст…
		!Госдума одобрила в первом чтении закон о коллекторах
	690) Хоккеисты ЦСКА победили «Магнитку» в матче Кубка Гагарина
		!Хоккеисты ЦСКА победили "Магнитку" в матче Кубка Гагарина
	31) "Ашан" вложит в 2016 г. в расширение бизнеса в России более 17 млрд руб. 
		!"Ашан" вложит 17 млрд рублей в открытие почти 50 магазинов в России
	854) Наибольший доход имеет замглавы администрации президента РФ Вячеслав Володин- 8